In [1]:
import numpy as np
from utils import *
import pandas as pd

In [2]:
data = pd.read_table("data/web_crowd.txt",header=None)

In [3]:
truth = pd.read_table("data/web_truth.txt", header=None)


In [4]:
def transform_data(data):
    X = np.array(data)
    rows, _ = X.shape
    n,m,k = np.max(np.array(data), axis=0)
    labels = np.zeros((m,n),dtype=np.int64)-1
    for r in range(rows):
        labels[X[r,1]-1,X[r,0]-1] = X[r,2]-1
    return labels


In [5]:
transform_data(data)


array([[ 4,  0,  0, ..., -1, -1, -1],
       [ 4, -1,  0, ...,  2,  1, -1],
       [ 4, -1, -1, ..., -1,  3, -1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]], dtype=int64)

In [6]:
n,m,k = np.max(np.array(data),axis=0)
print(n,m,k)

2665 177 5


In [7]:
labels = transform_data(data)


In [8]:
labels


array([[ 4,  0,  0, ..., -1, -1, -1],
       [ 4, -1,  0, ...,  2,  1, -1],
       [ 4, -1, -1, ..., -1,  3, -1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]], dtype=int64)

In [159]:
C_estimated = get_confusion_matrix(k,labels,groups=groups,sym=False)

[0 2 1 4 3] [0 2 1 4 3]
[[[0.16462867 0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]]

 [[0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]]

 [[0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]]]
[[[0.16462867 0.         0.         0.         0.        ]
  [0.         0.         0

In [ ]:
def get_M2(Z):
    _, n, k = Z.shape
    for a,b,c in [(1,2,0),(2,0,1),(0,1,2)]:
        Za = np.kron(Z[c,:,:],Z[b,:,:])@np.linalg.inv()

In [12]:
data[data[1]==1]

,0,1,2
0,1,1,5
6,2,1,1
12,3,1,1
18,4,1,1
24,5,1,2
...,...,...,...
15477,2651,1,5
15489,2653,1,4
15501,2655,1,5
15525,2659,1,3


In [13]:
truth

,0,1
0,1,5
1,2,1
2,3,1
3,4,1
4,5,2
...,...,...
2648,2661,5
2649,2662,2
2650,2663,4
2651,2664,3


In [14]:
from sklearn.metrics import confusion_matrix


In [15]:
truth_new = truth[truth[0].isin(data[data[1] == 2][0])]
pred_new = data[data[1] == 2][data[data[1] == 2][0].isin(truth_new[0])]


In [16]:
truth_new


,0,1
0,1,5
2,3,1
4,5,2
9,10,5
11,12,1
...,...,...
2630,2643,5
2636,2649,5
2638,2651,5
2650,2663,4


In [17]:
pred_new

,0,1,2
1,1,2,5
13,3,2,1
25,5,2,2
52,10,2,5
64,12,2,1
...,...,...,...
15430,2643,2,5
15467,2649,2,5
15478,2651,2,2
15549,2663,2,3


In [148]:
C = confusion_matrix(pred_new[2],truth_new[1])

In [149]:
C / np.sum(C,axis=0)[np.newaxis,:]

array([[0.84946237, 0.19565217, 0.07327586, 0.01986755, 0.02352941],
       [0.12903226, 0.7173913 , 0.10775862, 0.11258278, 0.04705882],
       [0.        , 0.07971014, 0.75862069, 0.25165563, 0.06470588],
       [0.        , 0.00724638, 0.04310345, 0.54304636, 0.04117647],
       [0.02150538, 0.        , 0.01724138, 0.07284768, 0.82352941]])

In [168]:
np.round(C_estimated[1],5)

array([[0.73485, 0.18811, 0.08929, 0.     , 0.09338],
       [0.16505, 0.56417, 0.10371, 0.08306, 0.11645],
       [0.     , 0.14277, 0.807  , 0.3148 , 0.07808],
       [0.05097, 0.06769, 0.     , 0.46681, 0.     ],
       [0.04913, 0.03726, 0.     , 0.13532, 0.71209]])

In [226]:
m, n = labels.shape


In [227]:
np.random.seed(0)
groups = np.random.randint(3, size=m)


In [9]:
groups = np.array([i%3 for i in range(m)])

In [10]:
groups


array([0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0,
       1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1,
       2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2,
       0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0,
       1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1,
       2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2,
       0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0,
       1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1,
       2])

In [23]:
Zg = get_Zg(k, labels, groups)


In [250]:
Zg

array([[[0.        , 0.        , 0.        , 0.        , 0.03389831],
        [0.05084746, 0.        , 0.        , 0.        , 0.        ],
        [0.03389831, 0.        , 0.        , 0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , 0.01694915, 0.        ],
        [0.        , 0.        , 0.01694915, 0.        , 0.        ],
        [0.        , 0.        , 0.01694915, 0.01694915, 0.        ]],

       [[0.        , 0.        , 0.        , 0.        , 0.03389831],
        [0.01694915, 0.        , 0.        , 0.        , 0.        ],
        [0.03389831, 0.        , 0.        , 0.        , 0.01694915],
        ...,
        [0.        , 0.        , 0.01694915, 0.01694915, 0.03389831],
        [0.        , 0.01694915, 0.        , 0.01694915, 0.        ],
        [0.        , 0.        , 0.        , 0.03389831, 0.01694915]],

       [[0.        , 0.        , 0.        , 0.        , 0.03389831],
        [0.03389831, 0.        , 0.        , 0.        , 0. 

In [251]:
Z = np.zeros((m,n,k))

In [252]:
for i in range(len(data)):
    Z[data.iloc[i, 1]-1, data.iloc[i, 0]-1, data.iloc[i, 2]-1] = 1


In [253]:
Zg = np.zeros((3,n,k))
for g in range(3):
    Zg[g,:,:] = np.mean(Z[groups==g,:,:],axis=0)

In [254]:
np.all(Zg == get_Zg(k, labels, groups))

True

In [24]:
M2s, M3s = get_M(Zg)


In [25]:
a,b,c = 1,2,0
Za = (Zg[c, :, :].T@Zg[b, :, :]/n)@np.linalg.inv(Zg[a, :, :].T @
                                                 Zg[b, :, :]/n)@Zg[a, :, :].T
Zb = (Zg[c, :, :].T@Zg[a, :, :]/n)@np.linalg.inv(Zg[b, :, :].T @
                                                 Zg[a, :, :]/n)@Zg[b, :, :].T

In [26]:
M3 = np.einsum("ji,ki,li->jkl", Za, Zb, Zg[c, :, :].T)/n
M2 = Za@Zb.T/n


In [27]:
u, s, vh = np.linalg.svd(M2, full_matrices=False)


In [28]:
get_whiten(M2, sym=True)


array([[  -9.19524314,   25.8762038 ,   59.94033902, -129.54118602,
          15.06207194],
       [ -13.53678217,   41.70697782,   56.48399218,   91.12466057,
        -107.97827229],
       [ -16.47236907,   42.96515193,  -22.52902475,   29.53756633,
         206.2432776 ],
       [ -20.49160929,   31.83289851,  -74.2551731 ,  -42.95577196,
        -130.40987372],
       [ -46.57817747,  -46.42863966,   12.38642101,    7.54231364,
          12.84231721]])

In [29]:
Q = u@np.diag(s**-0.5)

In [30]:
Q.shape

(5, 5)

In [31]:
M3_whiten = whiten_tensor(M3, Q)

In [287]:
M3_new = np.zeros((k,k,k))
for a in range(k):
    for b in range(k):
        for c in range(k):
            for d in range(k):
                for e in range(k):
                    for f in range(k):
                        M3_new[a,b,c]+=M3[d,e,f]*Q[d,a]*Q[e,b]*Q[f,c]

In [336]:
values, vectors = robust_tensor_power(M3_whiten, 20, 100)


In [292]:
np.allclose(M3_new,M3_whiten)

True

In [268]:
M3_2 = np.zeros((k,k,k))

In [269]:
for i1 in range(k):
    for i2 in range(k):
        for i3 in range(k):
            M3_2[i1, i2, i3] = np.sum(Za[i1,:]*Zb[i2,:]*Zg[c,:,i3])/n


In [275]:
M3_2


array([[[1.06549972e-06, 4.00909567e-07, 1.86179973e-07, 6.24276453e-08,
         1.03600769e-07],
        [2.54174415e-07, 4.46648428e-07, 1.40646002e-07, 7.01520550e-08,
         1.77188614e-07],
        [2.36062713e-07, 1.11559407e-07, 1.74379658e-07, 1.00333511e-07,
         1.17523448e-07],
        [5.52842310e-08, 6.02824313e-08, 1.31176047e-07, 2.14339296e-08,
         7.79550528e-08],
        [5.39319191e-08, 1.07918096e-07, 1.30878074e-07, 1.41321823e-07,
         2.78167546e-07]],

       [[3.37570139e-07, 3.50205452e-07, 1.47750442e-07, 1.00946691e-07,
         1.26257788e-07],
        [3.57496434e-07, 1.13783132e-06, 5.66940033e-07, 2.62104108e-07,
         1.53713051e-07],
        [1.10026953e-07, 4.33278892e-07, 4.48427205e-07, 2.72991611e-07,
         1.78693951e-07],
        [8.98954393e-08, 2.25697396e-07, 1.88496372e-07, 1.36317728e-07,
         1.52921146e-07],
        [1.52980207e-07, 2.17240718e-07, 1.19820034e-07, 7.11922876e-08,
         3.12528959e-07]],

      

In [276]:
M3

array([[[1.06549972e-06, 4.00909567e-07, 1.86179973e-07, 6.24276453e-08,
         1.03600769e-07],
        [2.54174415e-07, 4.46648428e-07, 1.40646002e-07, 7.01520550e-08,
         1.77188614e-07],
        [2.36062713e-07, 1.11559407e-07, 1.74379658e-07, 1.00333511e-07,
         1.17523448e-07],
        [5.52842310e-08, 6.02824313e-08, 1.31176047e-07, 2.14339296e-08,
         7.79550528e-08],
        [5.39319191e-08, 1.07918096e-07, 1.30878074e-07, 1.41321823e-07,
         2.78167546e-07]],

       [[3.37570139e-07, 3.50205452e-07, 1.47750442e-07, 1.00946691e-07,
         1.26257788e-07],
        [3.57496434e-07, 1.13783132e-06, 5.66940033e-07, 2.62104108e-07,
         1.53713051e-07],
        [1.10026953e-07, 4.33278892e-07, 4.48427205e-07, 2.72991611e-07,
         1.78693951e-07],
        [8.98954393e-08, 2.25697396e-07, 1.88496372e-07, 1.36317728e-07,
         1.52921146e-07],
        [1.52980207e-07, 2.17240718e-07, 1.19820034e-07, 7.11922876e-08,
         3.12528959e-07]],

      

In [39]:
def robust_tensor_power(T, L=20, N=100, sym=True):
    k, _, _ = T.shape
    if sym:
        T = sym_tensor(T)
    values = np.zeros(k)
    vectors = np.zeros((k, k))
    for h in range(k):
        theta_tau = []
        for tau in range(L):
            theta = np.random.randn(k)
            theta /= np.linalg.norm(theta)
            for t in range(N):
                theta = np.einsum("aef,e,f->a", T, theta, theta)
                theta /= np.linalg.norm(theta)
            theta_tau.append(theta)

        lam_best = float("-Inf")
        theta_best = np.array([])
        for theta in theta_tau:
            lam = np.einsum("efg,e,f,g->", T, theta, theta, theta)
            if lam > lam_best:
                lam_best = lam
                theta_best = theta

        theta = theta_best
        for t in range(N):
            theta = np.einsum("aef,e,f->a", T, theta, theta)
            theta /= np.linalg.norm(theta)
        lam = np.einsum("efg,e,f,g->", T, theta, theta, theta)

        values[h] = lam
        vectors[:, h] = theta
        T = T-lam*np.einsum("a,b,c->abc", theta, theta, theta)
    return values, vectors

In [133]:
lam, vec = robust_tensor_power(M3_whiten)

In [132]:
T = sym_tensor(M3_whiten)


In [121]:
lam, vec = robust_tensor_power(T, L = 20,N=100, sym=False)


In [122]:
temp = np.zeros((k, k, k))
for i in range(k):
    temp += lam[i]*np.einsum("a,b,c->abc", vec[:, i], vec[:, i], vec[:, i])

In [123]:
np.mean(np.abs(temp-T))

0.03568052611212279

In [134]:
mu = np.linalg.inv(Q.T)@vec@np.diag(lam)


In [165]:
def get_confusion_matrix(k, labels, groups=None, sym=True,cutoff=1e-7, L=20, N=100, seed=0):
    m, n = labels.shape
    if groups is None:
        np.random.seed(seed)
        groups = np.random.randint(3, size=m)
    Zg = get_Zg(k, labels, groups)
    M2s, M3s = get_M(Zg)
    Cc = np.zeros((3, k, k))
    W = np.zeros((3, k, k))
    for g, (M2, M3) in enumerate(zip(M2s, M3s)):
        Q = get_whiten(M2, sym)
        M3_whiten = whiten_tensor(M3, Q)
        values, vectors = robust_tensor_power(M3_whiten, L, N, sym)
        w = values**-2
        mu = np.linalg.inv(Q.T)@vectors@np.diag(values)
        best = np.argmax(mu, axis=0)
        for h in range(k):
            Cc[g, :, best[h]] = mu[:, h]
            W[g, best[h], best[h]] = w[h]
    W = np.mean(W, axis=0)
    C = np.zeros((m, k, k))
    for i in range(m):
        Ca = (np.sum(Cc, axis=0)-Cc[groups[i], :, :])/2
        Za = (np.sum(Zg, axis=0)-Zg[groups[i], :, :])/2
        E = np.zeros((k, k))
        for j in range(n):
            if labels[i,j]!=-1:
                E[labels[i, j], :] += Za[j, :]
        E /= n
        Ci = E@np.linalg.inv(W@Ca.T)
        if cutoff:
            Ci[Ci < cutoff] = cutoff
        colsums = np.sum(Ci, axis=0)
        Ci /= colsums[np.newaxis, :]
        C[i, :, :] = Ci
    return C


In [166]:
C_estimated = get_confusion_matrix(k, labels, groups=groups, sym=False)


In [136]:
np.argmax(mu,axis=1)

array([0, 2, 1, 4, 3], dtype=int64)

In [138]:
mu0 = lam[0]*np.linalg.inv(Q.T)@vec[:,0]


In [161]:
E = -np.eye(5)

In [163]:
E[E<0]=0

In [164]:
E

array([[ 0., -0., -0., -0., -0.],
       [-0.,  0., -0., -0., -0.],
       [-0., -0.,  0., -0., -0.],
       [-0., -0., -0.,  0., -0.],
       [-0., -0., -0., -0.,  0.]])

In [139]:
np.argmax(mu0)

0

In [82]:
robust_tensor_power(temp,sym=False)


(array([2.47221395, 1.9597989 , 1.68658776, 1.62258932, 1.5157169 ]),
 array([[-0.20816801, -0.45904245, -0.28248499, -0.78507705, -0.44330809],
        [ 0.34572185,  0.49387216,  0.52550452, -0.60544145,  0.20886257],
        [ 0.44722585, -0.25200777,  0.59235523,  0.1237719 , -0.73952102],
        [-0.78471769,  0.16964156,  0.39568989,  0.03355718, -0.24202772],
        [ 0.14611525,  0.67311525, -0.36958358, -0.02547738, -0.39291933]]))

In [83]:
robust_tensor_power(M3_whiten,sym=False)


(array([2.46460466, 1.95134792, 1.69621225, 1.61947364, 1.52595186]),
 array([[-0.19890427, -0.44754347, -0.27192651, -0.7790689 , -0.44857849],
        [ 0.33327075,  0.49026603,  0.52012714, -0.61032911,  0.20734864],
        [ 0.44301318, -0.23003884,  0.60929923,  0.1369673 , -0.7439836 ],
        [-0.79524795,  0.1743316 ,  0.39476733,  0.03790537, -0.24151606],
        [ 0.14383224,  0.68995267, -0.35838099, -0.01879279, -0.37939724]]))

In [84]:
temp-M3_whiten

array([[[-1.08502728e-01,  1.05916157e-01, -6.35574630e-02,
         -3.43508189e-02,  5.84778878e-02],
        [ 1.96681210e-02, -6.98760455e-02, -3.20724535e-02,
         -5.50540447e-05, -1.21747069e-01],
        [-8.06535108e-02, -6.56195518e-02, -7.87530114e-02,
         -3.46972306e-02,  4.28376898e-02],
        [-3.38258278e-02,  2.33976404e-02,  2.84785558e-02,
          6.85807628e-02,  6.02308420e-02],
        [ 3.13195419e-02, -6.54631363e-02,  8.33227681e-02,
          5.97476890e-02,  2.93345015e-01]],

       [[ 1.08825240e-01, -4.38449374e-02, -2.27620711e-02,
          7.12558959e-02, -1.67551172e-01],
        [-1.07955969e-01,  4.71086683e-02,  1.63862581e-01,
         -1.08804478e-01, -9.55043605e-02],
        [-5.15489931e-02,  3.80441493e-03,  1.11494404e-01,
         -2.27938399e-02, -3.64960660e-02],
        [ 4.60967670e-02, -1.14286794e-01,  3.20545504e-02,
         -1.90397928e-01, -9.42836059e-02],
        [-1.10369938e-01,  7.54648987e-02,  4.86100648e-02,
  

In [71]:
M3_whiten-temp

array([[[-6.25658403e-03, -2.08701701e-02,  6.95101083e-03,
          2.98214755e-04, -3.07195298e-02],
        [ 6.53778655e-02,  6.29059032e-03,  7.11755865e-03,
          3.11941863e-02, -2.07323904e-03],
        [ 2.40470587e-02,  4.06646570e-02, -7.25391499e-02,
          9.06341042e-02,  2.77452727e-02],
        [-2.26776426e-04,  7.74149191e-03,  2.74583178e-02,
         -1.08783679e-01, -6.30078602e-02],
        [-3.56118387e-03, -5.83571718e-02, -1.27398055e-02,
         -6.25247072e-02, -2.37728792e-01]],

       [[-2.37792535e-02, -1.97405178e-02, -2.19282371e-03,
         -4.01167636e-02,  4.37308640e-02],
        [ 4.43705142e-02, -6.07402112e-02, -1.61836424e-01,
          4.48495643e-02,  1.65599629e-01],
        [ 2.65940983e-02, -1.77825783e-03, -6.85812595e-02,
         -3.97116078e-02, -9.42294362e-02],
        [-1.49576348e-02,  5.03318796e-02, -9.45599982e-02,
          9.85592661e-02,  1.54445472e-01],
        [-1.34503702e-02, -5.36963022e-03, -1.79335567e-01,
  